## 심플한 Agent 생성 및 실행하기

#### .env 파일 로드

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

#### 에이전트 생성

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are good at telling jokes who responds in korean.",
    name="Joker"
)

## 참조 : https://github.com/warnov/ms-agent-framework-step-by-step-workshop
## endpoint와 deployment_name을 .env에 설정된 기본값이 아닌 환경 변수에서 읽도록 수정하려면 다음과 같이 변경
## 기본값 Key는 AZURE_OPENAI_ENDPOINT와 AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# agent = AzureOpenAIChatClient(
#    credential=AzureCliCredential(),
#    endpoint=os.environ["AOAI_ENDPOINT"],
#    deployment_name=os.environ["AOAI_DEPLOYMENT"]
#).as_agent(
#    instructions="You are good at telling tales.",
#    name="Joker"
#)

#### 에이전트 실행

에이전트를 실행하려면 에이전트 인스턴스에서 run 메서드를 호출하고 사용자 입력을 제공하십시오. 에이전트는 응답 객체를 반환하며, result.text 속성에 접근하면 에이전트가 반환한 텍스트 결과를 확인할 수 있습니다.

In [ ]:
async def main():
    result = await agent.run("Tell me a joke about a pirate.")
    print(result.text)

await main()

#### 스트리밍을 사용하여 에이전트 실행

스트리밍 방식으로 에이전트를 실행하려면 에이전트 인스턴스의 run_stream 메서드를 호출하고 사용자 입력을 제공하십시오. 에이전트는 update 객체 목록을 스트리밍 방식으로 전송하며, 각 업데이트 객체의 text 속성에 접근하면 해당 업데이트에 포함된 텍스트 결과의 일부를 얻을 수 있습니다.

In [ ]:
async def main():
    async for update in agent.run_stream("Tell me a long joke about a pirate."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

await main()

#### ChatMessage를 사용하여 에이전트 실행 (이미지 포함)

단순 문자열 대신 하나 이상의 ChatMessage 객체를 run 및 run_stream 메서드에 제공할 수도 있습니다. 예제는 온라인 이미지를 사용하여 ChatMessage 객체를 생성하는 방법을 보여줍니다.

In [ ]:
from agent_framework import ChatMessage, Content, Role

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text(text="Tell me a joke about this image?"),
        Content.from_uri(uri="https://www.fotosanimales.es/wp-content/uploads/2017/12/pinguino.jpg", media_type="image/jpeg")
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()

DataContent를 사용하여 로컬 파일 시스템에서 이미지를 불러오는 방법도 있습니다

In [ ]:
from agent_framework import ChatMessage, Content, Role

# Load image from local file
with open("images/image1.jpg", "rb") as f:
    image_bytes = f.read()

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text(text="What do you see in this image?"),
        Content.from_data(
            data=image_bytes,
            media_type="image/jpeg"
        )
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()